In [1]:
import pandas as pd
import os

In [2]:
! pip install matplotlib
! pip install seaborn
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
reporting_folder=r'C:\Users\JenishaE\Documents\finalDE\finalProject-Data\reporting'

In [4]:
emp_dim = os.path.join(reporting_folder, 'emp_dim.csv')
emp_training_performance_fact=os.path.join(reporting_folder,'emp_training_performance_fact.csv')
job_performance_dim=os.path.join(reporting_folder,'job_performance_dim.csv')
skill_dim=os.path.join(reporting_folder,'skill_dim.csv')
training_dim=os.path.join(reporting_folder,'training_dim.csv')
emp_skill_fact=os.path.join(reporting_folder,'emp_skill_fact.csv')
emp_training_performance_fact=os.path.join(reporting_folder,'emp_training_performance_fact.csv')

**Count of Skills Per Employee**

In [5]:
emp_dim = pd.read_csv(emp_dim)
emp_training_performance_fact = pd.read_csv(emp_training_performance_fact)
job_performance_dim = pd.read_csv(job_performance_dim)
skill_dim = pd.read_csv(skill_dim)
training_dim = pd.read_csv(training_dim)
emp_skill_fact = pd.read_csv(emp_skill_fact)

In [6]:
empskill_emp_merged_df = pd.merge(emp_skill_fact, emp_dim, on='emp_id', how='right')

# Calculate the number of unique skills per employee
skills_per_employee = empskill_emp_merged_df.groupby(['emp_id', 'emp_name'])['skill_id'].nunique().reset_index(name='no_of_skills')

In [7]:
skills_per_employee.head()

,emp_id,emp_name,no_of_skills
0,E001,Sharon Burns,2
1,E002,Hannah Sanchez,1
2,E003,Larry Waters,1
3,E004,Angela Delacruz,0
4,E005,John Cook,2


In [8]:
skills_per_employee.shape

(5000, 3)

**Top Project and Hackerrank score in each training**

In [9]:
merged_df = pd.merge(emp_training_performance_fact, emp_dim, on='emp_id', how='right')
merged_df = pd.merge(merged_df, training_dim, on='T_id', how='right')

In [10]:
top_scores_project = merged_df.groupby('T_name')['project_score'].max().reset_index()
top_scores_hackerrank = merged_df.groupby('T_name')['hackerrank_score'].max().reset_index()

In [11]:
top_scores_df = pd.merge(top_scores_project, top_scores_hackerrank, on='T_name')

In [12]:
top_scores_df.head()

,T_name,project_score,hackerrank_score
0,Advanced Data Visualization,100.0,100.0
1,Advanced JavaScript,100.0,100.0
2,Business Process Management,99.0,100.0
3,Client Relationship Management,99.0,100.0
4,Consulting Essentials,100.0,100.0


In [13]:
top_scores_df.shape

(15, 3)

**Employee Count by Skill Category**

In [14]:
merged_df = pd.merge(emp_skill_fact, skill_dim, on='skill_id', how='right')

In [15]:
skill_category_counts = merged_df.groupby('skill_category')['emp_id'].nunique().reset_index()

In [16]:
skill_category_counts.columns = ['skill_category', 'employee_count']

In [17]:
skill_category_counts.head()

,skill_category,employee_count
0,consulting,1609
1,data,1539
2,full_stack,1619


**Count of Employees who have done certifications in each skill**

In [18]:
certified_employees_df = emp_skill_fact[emp_skill_fact['certifications'] == 'Yes']


In [19]:
merged_df = pd.merge(certified_employees_df, skill_dim, on='skill_id', how='right')

In [20]:
certification_counts = merged_df.groupby('skill_name')['emp_id'].nunique().reset_index()

In [21]:
certification_counts.columns = ['skill_name', 'certified_employee_count']

In [22]:
certification_counts.head()

,skill_name,certified_employee_count
0,Big Data,148
1,Client Management,169
2,Consulting Basics,193
3,Data Visualization,187
4,Docker,187


**Average Project Score of Each Training**

In [23]:
merged_df = emp_training_performance_fact.merge(training_dim, on='T_id', how='right')

In [24]:
average_project_score = merged_df.groupby('T_name')['project_score'].mean().reset_index()

# Rename columns for clarity
average_project_score.columns = ['T_name', 'average_project_score']

In [25]:
average_project_score.head()

,T_name,average_project_score
0,Advanced Data Visualization,37.448052
1,Advanced JavaScript,37.629944
2,Business Process Management,38.165919
3,Client Relationship Management,35.379592
4,Consulting Essentials,36.043478


**Count of Completion and not completion of each training**

In [26]:
merged_df = pd.merge(emp_training_performance_fact, training_dim, on='T_id',how='right')

In [27]:
training_completion_counts = merged_df.groupby('T_name')['completed'].value_counts().unstack(fill_value=0)

In [28]:
training_completion_counts.columns = ['not_completed', 'completed']

# Reset index to have a clean DataFrame
training_completion_counts.reset_index(inplace=True)

# Display the resulting DataFrame
training_completion_counts.head()

,T_name,not_completed,completed
0,Advanced Data Visualization,233,229
1,Advanced JavaScript,166,188
2,Business Process Management,114,109
3,Client Relationship Management,121,124
4,Consulting Essentials,130,123


**Employee who have Maximum Project count in Each skill**

In [29]:
# Step 1: Merge emp_skill_fact with emp_dim to get employee names
emp_skill_with_names = pd.merge(emp_skill_fact, emp_dim, on='emp_id', how='left')

In [30]:
# Step 2: Merge with skill_dim to get skill names
emp_skill_with_names = pd.merge(emp_skill_with_names, skill_dim, on='skill_id', how='left')

In [31]:
# Check column names to ensure correctness
print("Columns in emp_skill_with_names DataFrame:")
print(emp_skill_with_names.columns)

Columns in emp_skill_with_names DataFrame:
Index(['emp_id', 'skill_id', 'proficiency_level',
       'no_of_projects_done_in_that_skill', 'certifications', 'emp_name',
       'department', 'designation', 'DOJ', 'age', 'experience', 'salary',
       'no_of_promotions', 'education', 'skill_name', 'skill_category'],
      dtype='object')


In [32]:
# Step 3: Clean column names by stripping whitespace
emp_skill_with_names.columns = emp_skill_with_names.columns.str.strip()


In [33]:
# Group by skill_name and emp_name to get the maximum project count
max_projects_per_skill = emp_skill_with_names.groupby(['skill_name', 'emp_name'])['no_of_projects_done_in_that_skill'].max().reset_index()

In [34]:
# Step 4: Find the maximum project count for each skill
max_projects_per_skill = max_projects_per_skill.loc[max_projects_per_skill.groupby('skill_name')['no_of_projects_done_in_that_skill'].idxmax()]

In [35]:
# Step 5: Rename columns for clarity
max_projects_per_skill.rename(columns={'no_of_projects_done_in_that_skill': 'max_project_count'}, inplace=True)

# Display the resulting DataFrame
max_projects_per_skill.head()

,skill_name,emp_name,max_project_count
8,Big Data,Amanda Fox,12
332,Client Management,Aaron Dunn,12
696,Consulting Basics,Amy Green,12
1088,Data Visualization,Allen Price,12
1446,Docker,Alison Rose,12



**Count of employees with proficiency greater than 7 in a training**

In [36]:
# Step 1: Merge emp_skill_fact with skill_dim to get skill names
emp_skill_with_names = pd.merge(emp_skill_fact, skill_dim, on='skill_id', how='right')


In [37]:
# Step 2: Merge with emp_dim to get employee names
emp_skill_with_names = pd.merge(emp_skill_with_names, emp_dim, on='emp_id', how='right')

In [38]:
# Step 3: Filter employees with proficiency greater than 7
filtered_emp_skills = emp_skill_with_names[emp_skill_with_names['proficiency_level'] > 7]

In [39]:
# Step 4: Count unique employees by skill_name
count_proficient_employees = filtered_emp_skills.groupby('skill_name')['emp_id'].nunique().reset_index()

In [40]:
# Step 5: Rename columns for clarity
count_proficient_employees.rename(columns={'emp_id': 'count_of_employees_with_proficiency_greater_than_7'}, inplace=True)

# Display the resulting DataFrame
count_proficient_employees.head()

,skill_name,count_of_employees_with_proficiency_greater_than_7
0,Big Data,114
1,Client Management,114
2,Consulting Basics,121
3,Data Visualization,109
4,Docker,101


**Skill Gap Analysis**

In [41]:
merged_df = pd.merge(emp_training_performance_fact, training_dim, on='T_id')

In [42]:
merged_df = pd.merge(merged_df, emp_dim, on='emp_id')

In [43]:
# Assuming 'project_score' is the column that contains the scores
# Calculate total employees per training
total_employees_per_training = merged_df.groupby('T_name')['emp_name'].count()

In [44]:
# Calculate employees scoring below and above 40
below_40_counts = merged_df[merged_df['project_score'] < 40].groupby('T_name')['emp_name'].count()
above_40_counts = merged_df[merged_df['project_score'] >= 40].groupby('T_name')['emp_name'].count()

In [45]:
# Calculate percentages
percentage_below_40 = (below_40_counts / total_employees_per_training) * 100
percentage_above_40 = (above_40_counts / total_employees_per_training) * 100
percentage_below_40 = percentage_below_40.round(2)
percentage_above_40 = percentage_above_40.round(2)

In [46]:
summary_df = pd.DataFrame({
    'Percentage Below 40': percentage_below_40,
    'Percentage Above 40': percentage_above_40
}).reset_index()

# Display the summary DataFrame
summary_df.head()

,T_name,Percentage Below 40,Percentage Above 40
0,Advanced Data Visualization,64.50,35.50
1,Advanced JavaScript,62.15,37.85
2,Business Process Management,66.82,33.18
3,Client Relationship Management,67.35,32.65
4,Consulting Essentials,65.61,34.39
